# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();
# Cargamos las dataframes con las instancias
df_completos_random = pd.read_csv("instancias/completos-random/indice.csv");
df_parametros = pd.read_csv("instancias/parametros/indice.csv");
df_comparacion_heuristicas = pd.read_csv("instancias/comparacion-heuristicas/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- H1: Heurística de vecino más cercano.
- H2: Heurística de arista más corta.
- HAGM: Heurística de árbol generador mínimo.
- METAH1: Metaheurística con memoria de soluciones.
- METAH2: Metaheurística con memoria de aristas.

In [3]:
def correr_experimento(metodo, archivo_instancia, param):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    max_iter = str(param[0])
    max_iter_sm = str(param[1])
    max_time = str(param[2])
    mem_size = str(param[3])
    porc = str(param[4])
    process = subprocess.Popen(["../tsp", metodo, max_iter, max_iter_sm, max_time, mem_size, porc], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    # Leo el optimo
    optimo = float(process.stdout.readline().split(" ")[1])
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return (tiempo_de_ejecucion, optimo);

In [4]:
def correr_experimento_parametros(metodo, archivo_instancia, param):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    max_iter = str(param[0])
    max_iter_sm = str(param[1])
    max_time = str(param[2])
    mem_size = str(param[3])
    porc = str(param[4])
    process = subprocess.Popen(["../tsp", metodo, max_iter, max_iter_sm, max_time, mem_size, porc], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read())
    # Es el tiempo que tengo que leer en la celda
    optimo_encontrado = process.stdout.readline().split(" ")
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return float(optimo_encontrado[1]), tiempo_de_ejecucion; # JONA Devuelvo la tupla, con tiempo de ejecucion

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [5]:
experimentos = [];
experimentos_parametros = [];
experimentos_heuristicas= [];

## Experimento 1
Correr todos los experimentos en las primeras 30 instancias de completos-random.

In [32]:
for n in range(0, 50):
    fila_n = df_completos_random.iloc[n];
    experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "H1", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "H2", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "HAGM", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "METAH1", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "METAH2", fila_n["archivo"]]);

## Experimento 2
Correr Metaheurísticas para buscar "buenos" parámetros.

In [6]:
for n in range(0, df_parametros.shape[0]):
    fila_n = df_parametros.iloc[n];
    experimentos_parametros.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "METAH1", fila_n["archivo"], fila_n["instancia"]]);
    experimentos_parametros.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "METAH2", fila_n["archivo"], fila_n["instancia"]]);

## Experimento 3
Comparar heurísticas contra todos los 5 datasets conocidos.

In [34]:
for n in range(0, df_comparacion_heuristicas.shape[0]):
    fila_n = df_comparacion_heuristicas.iloc[n];
    experimentos_heuristicas.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "H1", fila_n["archivo"]]);
    experimentos_heuristicas.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "H2", fila_n["archivo"]]);
    experimentos_heuristicas.append([fila_n["dataset"], fila_n["n"], fila_n["W"], "HAGM", fila_n["archivo"]]);

## Ejecutar los experimentos de complejidad y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [9]:
columnas = ["dataset", "n", "W","metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).

#Parametros globales
param = [0] * 5
param[0] = 100000
param[1] = 10000
param[2] = 30000
param[3] = 50
param[4] = 0.1

for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[3], experimento[4], param)[0]);
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultados_complejidad.csv", index=False, header=True);

print("Listo")

'Experimento: 44/250'

KeyboardInterrupt: 

## Ejecutar los experimentos de parametros.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [9]:
columnas = ["dataset", "n", "W", "metodo", "optimo", "tiempo", "instancia"];
filas = [];
numero = 1

for experimento in experimentos_parametros:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_parametros)))
    numero += 1
    
    #Parametros globales
    param = [0] * 5
    param[0] = 100000
    param[1] = 10000
    param[2] = 30000
    param[3] = 50
    param[4] = 0.1
    
    # Cargamos parametro de test
    if experimento[0] == "maxiter":
        param[0] = experimento[1]
    elif experimento[0] == "maxiter-sm":
        param[1] = experimento[1]
    elif experimento[0] == "maxtime":
        param[2] = experimento[1]
    elif experimento[0] == "memsize":
        param[3] = experimento[1]
    elif experimento[0] == "porc":
        param[4] = experimento[1]
    else:
        print("Error de parametros")
        break
        
    # Ejecutamos el experimento.
    # JONA Aca deberia guardar el optimo y el tiempo, para que se agregue al csv
    optimo, tiempo = correr_experimento_parametros(experimento[3], experimento[4], param);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], optimo, tiempo, experimento[5]]);    
    
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultados_parametros.csv", index=False, header=True);

print("Listo")

'Experimento: 330/330'

Listo


## Ejecutar los experimentos de heuristicas

In [ ]:
columnas = ["dataset", "n", "W","metodo", "tiempo", "optimo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).

#Parametros globales
param = [0] * 5
param[0] = 100000
param[1] = 10000
param[2] = 30000
param[3] = 50
param[4] = 0.1

for experimento in experimentos_heuristicas:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_heuristicas)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tupla = correr_experimento(experimento[3], experimento[4], param)
        tiempos.append(tupla[0]);
    tiempo = np.median(tiempos);
    optimo = tupla[1]
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo, optimo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultados_heuristicas.csv", index=False, header=True);

print("Listo")